In [3]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [113]:
# data cleaning
admin=pd.read_excel(r"C:\Users\Dear User\OneDrive\桌面\Xelay\Admin.xlsx")
labs=pd.read_excel(r"C:\Users\Dear User\OneDrive\桌面\Xelay\Labs.xlsx")
print(admin.head())
print(labs.head())

C:\Users\Dear User\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


   ID    Med Admin Date  Units
0   1  Med A 2012-07-02   1500
1   1  Med A 2012-07-06   1500
2   1  Med A 2012-07-09   1500
3   1  Med A 2012-07-11   1500
4   1  Med A 2012-07-13   1500
    ID  DRAW_DATE LAB_RESULT_CODE LAB_VALUE
0  1.0 2012-07-04           LAB A       679
1  1.0 2012-08-01           LAB A       621
2  1.0 2012-09-05           LAB A       915
3  1.0 2012-10-03           LAB A       848
4  1.0 2012-11-07           LAB A       845


In [5]:
def info(data):
    info = pd.DataFrame()
    info["Isnull"] = data.isnull().sum()
    info.insert(1, "IsNa", data.isna().sum(), True)
    info.insert(2, "Duplicate", data.duplicated().sum(), True)
    info.insert(3, "Unique", data.nunique(), True)
    info.insert(4, "count", data.count(), True)
    return info
print(info(admin))
print(info(labs))


            Isnull  IsNa  Duplicate  Unique  count
ID               0     0          0     129   2022
Med              0     0          0       2   2022
Admin Date       0     0          0      96   2022
Units            0     0          0     111   2022
                 Isnull  IsNa  Duplicate  Unique  count
ID                  302   302          5     117   2095
DRAW_DATE             0     0          5     119   2397
LAB_RESULT_CODE       0     0          5       3   2397
LAB_VALUE             0     0          5     615   2397


In [6]:
#labs NA cleaning 
#Although there is 0.12% NA in all data, I drop this for the complete of the result
labs_new=labs.dropna()

print(labs_new)
print(info(labs_new))

         ID  DRAW_DATE LAB_RESULT_CODE LAB_VALUE
0       1.0 2012-07-04           LAB A       679
1       1.0 2012-08-01           LAB A       621
2       1.0 2012-09-05           LAB A       915
3       1.0 2012-10-03           LAB A       848
4       1.0 2012-11-07           LAB A       845
...     ...        ...             ...       ...
2392  129.0 2012-08-16           LAB B      10.0
2393  129.0 2012-09-04           LAB B       9.4
2394  129.0 2012-07-05           LAB C        36
2395  129.0 2012-08-02           LAB C        39
2396  129.0 2012-09-04           LAB C        27

[2095 rows x 4 columns]
                 Isnull  IsNa  Duplicate  Unique  count
ID                    0     0          0     117   2095
DRAW_DATE             0     0          0     112   2095
LAB_RESULT_CODE       0     0          0       3   2095
LAB_VALUE             0     0          0     560   2095


In [126]:
#Q1 
admin_Q1=admin.copy()
admin_Q1["Admin Date"]=admin_Q1["Admin Date"].dt.to_period("M")
sum_med_m=admin_Q1.groupby(["Admin Date","Med"])["Units"].sum()
print(sum_med_m)

#Q2
admin_Q2=admin_Q1.copy()
admin_Q2=admin_Q2[(admin_Q2["Admin Date"]>="2012-07") & (admin_new["Admin Date"]<="2012-12")]
count_patient=admin_Q2.groupby("Med")["ID"].nunique()
print(count_patient)

Admin Date  Med  
2012-07     Med A    4303700
2012-08     Med A    4477100
2012-09     Med A     849900
            Med B        535
2012-10     Med B        393
2012-11     Med A      75300
            Med B        420
2012-12     Med A      10200
            Med B          1
Name: Units, dtype: int64
Med
Med A    113
Med B     97
Name: ID, dtype: int64


In [8]:
#Q3 7~11
admin_new=admin.copy()
admin_new["Admin Date"]=admin_new["Admin Date"].dt.to_period("M")
total_dose= admin_new.groupby(["Admin Date","Med"])["Units"].sum()
patient_count=admin_new.groupby(["Admin Date","Med"])["Units"].count()

print(info(admin_new))
print(total_dose)
print(patient_count)

result_Q3=total_dose/patient_count
result_Q3=round(result_Q3,2)

print(result_Q3)

            Isnull  IsNa  Duplicate  Unique  count
ID               0     0       1435     129   2022
Med              0     0       1435       2   2022
Admin Date       0     0       1435       6   2022
Units            0     0       1435     111   2022
Admin Date  Med  
2012-07     Med A    4303700
2012-08     Med A    4477100
2012-09     Med A     849900
            Med B        535
2012-10     Med B        393
2012-11     Med A      75300
            Med B        420
2012-12     Med A      10200
            Med B          1
Name: Units, dtype: int64
Admin Date  Med  
2012-07     Med A    839
2012-08     Med A    793
2012-09     Med A    160
            Med B     76
2012-10     Med B     68
2012-11     Med A      7
            Med B     75
2012-12     Med A      3
            Med B      1
Name: Units, dtype: int64
Admin Date  Med  
2012-07     Med A     5129.56
2012-08     Med A     5645.78
2012-09     Med A     5311.88
            Med B        7.04
2012-10     Med B        5.78
201

In [228]:
#Q4 9~11 三種可能性 1.A->B 2.A->A 3.B->B
admin_new_Q4=admin_new.copy()
admin_new_Q4=admin_new_Q4[(admin_new_Q4["Admin Date"]>="2012-9") & (admin_new_Q4["Admin Date"]<"2012-12")]
print(admin_new_Q4)
print(info(admin_new_Q4))

switched_patients = set()  
new_Med_B_patients = set()  
Med_A_patients = set()  

for patient_id in admin_new_Q4["ID"].unique():
    patient_data = admin_new_Q4[admin_new_Q4["ID"] == patient_id]
    meds_used = set(patient_data["Med"])

    if "Med A" in meds_used and "Med B" in meds_used:
        switched_patients.add(patient_id)
    elif "Med B" in meds_used and "Med A" not in meds_used:
        new_Med_B_patients.add(patient_id)
    elif "Med A" in meds_used and "Med B" not in meds_used:
        Med_A_patients.add(patient_id)

switched_count = len(switched_patients)
new_Med_B_count = len(new_Med_B_patients)
Med_A_patients_count = len(Med_A_patients)

print("Switched Patients:", switched_count)
print("New Med B Patients:", new_Med_B_count)
print("Med A Patients:", Med_A_patients_count)
print(len(switched_patients))

       ID    Med Admin Date  Units
24      1  Med A    2012-09   1900
25      1  Med A    2012-09   1900
26      1  Med B    2012-09      4
27      1  Med B    2012-10      3
28      1  Med B    2012-11      1
...   ...    ...        ...    ...
2009  128  Med B    2012-10      1
2010  128  Med B    2012-11      2
2019  129  Med A    2012-09   5200
2020  129  Med A    2012-09   6500
2021  129  Med B    2012-09     10

[386 rows x 4 columns]
            Isnull  IsNa  Duplicate  Unique  count
ID               0     0         85     107    386
Med              0     0         85       2    386
Admin Date       0     0         85       3    386
Units            0     0         85      73    386
Switched Patients: 67
New Med B Patients: 30
Med A Patients: 10
67


In [243]:
#Q4 轉換為def定義式 以重複利用

switched_patients_other = set()
new_Med_B_patients_other = set()
Med_A_patients_other= set()

def calculate_patient_stats(admin_data, start_date, end_date):
    admin_filtered = admin_data[(admin_data["Admin Date"] >= start_date) & (admin_data["Admin Date"] < end_date)]

    for patient_id in admin_filtered["ID"].unique():
        patient_data = admin_filtered[admin_filtered["ID"] == patient_id]
        meds_used = set(patient_data["Med"])

        if "Med A" in meds_used and "Med B" in meds_used:
            switched_patients_other.add(patient_id)
        elif "Med B" in meds_used and "Med A" not in meds_used:
            new_Med_B_patients_other.add(patient_id)
        elif "Med A" in meds_used and "Med B" not in meds_used:
            Med_A_patients_other.add(patient_id)

    switched_count_other = len(switched_patients_other)
    new_Med_B_count_other = len(new_Med_B_patients_other)
    Med_A_patients_count_other = len(Med_A_patients_other)

    return {
        "Switched Patients": switched_count_other,
        "New Med B Patients": new_Med_B_count_other,
        "Med A Patients": Med_A_patients_count_other
    }


start_date = "2012-9"
end_date = "2012-10"
result = calculate_patient_stats(admin_new, start_date, end_date)
print("Results for the selected time period:")
for key, value in result.items():
    print(f"{key}: {value}")

print(switched_patients_other,new_Med_B_patients_other,Med_A_patients_other)

Results for the selected time period:
Switched Patients: 62
New Med B Patients: 14
Med A Patients: 11
{128, 1, 129, 4, 8, 9, 10, 12, 15, 16, 17, 18, 19, 20, 21, 25, 29, 30, 38, 39, 46, 47, 48, 49, 52, 53, 54, 60, 65, 69, 73, 75, 81, 82, 83, 85, 86, 87, 90, 91, 92, 94, 96, 97, 98, 99, 100, 101, 104, 105, 106, 110, 112, 113, 114, 116, 117, 118, 119, 122, 125, 126} {2, 36, 5, 41, 123, 77, 109, 24, 57, 56, 88, 28, 63, 31} {64, 33, 66, 74, 14, 111, 80, 50, 22, 89, 62}


In [250]:
#Q5 9~11 轉換前Med A的平均使用期間
#print("Switched Patients:", switched_patients)
#print("New Med B Patients:", new_Med_B_patients)
#print("Med A Patients:", Med_A_patients)

switched_list=list(switched_patients)
admin_new_Q5=admin.copy()
admin_new_Q5=admin_new_Q5[(admin_new_Q5["Admin Date"]>="2012-9") & (admin_new_Q5["Admin Date"]<"2012-12")]
admin_new_Q5=admin_new_Q5[admin_new_Q5["ID"].isin(switched_list)]

#print(admin_new_Q5)
#print(info(admin_new_Q5))

diff=[]
for patient_id in range(1, 130):
    med_A_data = admin_new_Q5[(admin_new_Q5["Med"] == "Med A") & (admin_new_Q5["ID"] == patient_id)]

    if not med_A_data.empty:
        min_date = med_A_data["Admin Date"].min()
        max_date = med_A_data["Admin Date"].max()
        difference = max_date - min_date
        diff.append(difference)
        
total_difference=sum(diff,pd.Timedelta(0))
duration=7*len(diff)
averange_week=total_difference/duration
print(len(diff))
print(averange_week)
print(total_difference)

67
0 days 09:49:30.575692963
192 days 00:00:00


In [251]:
#Q6

admin_new_Q6=admin_new_Q5.copy()
sum_dose=admin_new_Q6.groupby("Med")["Units"].sum()
sum_dose_A=sum_dose["Med A"]
print(admin_new_Q6)
print(sum_dose_A)
print(sum_dose)


averange_dose_A=sum_dose_A/(len(diff)*6.33)
print(averange_dose_A)

print(admin_new_Q6["ID"].nunique())

       ID    Med Admin Date  Units
24      1  Med A 2012-09-03   1900
25      1  Med A 2012-09-05   1900
26      1  Med B 2012-09-12      4
27      1  Med B 2012-10-17      3
28      1  Med B 2012-11-28      1
...   ...    ...        ...    ...
2009  128  Med B 2012-10-11      1
2010  128  Med B 2012-11-15      2
2019  129  Med A 2012-09-04   5200
2020  129  Med A 2012-09-06   6500
2021  129  Med B 2012-09-13     10

[312 rows x 4 columns]
792300
Med
Med A    792300
Med B      1064
Name: Units, dtype: int64
1868.1474145858385
67


In [253]:
med_b_data = admin_new_Q5[admin_new_Q5["Med"] == "Med B"]
med_b_count=med_b_data.groupby("ID")["Med"].count()
print(med_b_data)
print(med_b_count.head(67))

single_use_med_b = med_b_count[med_b_count == 1].index
multiple_use_med_b=med_b_count[med_b_count > 1].index
print(len(single_use_med_b))
print(len(multiple_use_med_b))
#已知有兩種可能用量1次 或一次以上

count=0
difference_b=[]
if med_b_data["ID"].isin(multiple_use_med_b).any():
    max_date_b=med_b_data["Admin Date"].max()
    min_date_b=med_b_data["Admin Date"].min()
    diff=max_date_b-min_date_b
    difference_b.append(diff)

print(difference_b)
# 84天(使用量1次:6個、多次:78天)
averange_month=84/(4.33*7)
averange_dose_B=sum_dose["Med B"]/(averange_month+len(med_b_count))
print(averange_dose_B)

       ID    Med Admin Date  Units
26      1  Med B 2012-09-12      4
27      1  Med B 2012-10-17      3
28      1  Med B 2012-11-28      1
82      4  Med B 2012-09-12      3
83      4  Med B 2012-11-14      1
...   ...    ...        ...    ...
1979  126  Med B 2012-11-14      2
2008  128  Med B 2012-09-25      1
2009  128  Med B 2012-10-11      1
2010  128  Med B 2012-11-15      2
2021  129  Med B 2012-09-13     10

[163 rows x 4 columns]
ID
1      3
4      2
8      3
9      3
10     3
      ..
122    2
125    3
126    2
128    3
129    1
Name: Med, Length: 67, dtype: int64
6
61
[Timedelta('78 days 00:00:00')]
15.249809671973784
67


In [254]:
#Q7 
admin_new_Q7=admin_new_Q6.copy()
sum_dose=admin_new_Q7.groupby("Med")["Units"].sum()
print(sum_dose)
breakeven_price_B=sum_dose["Med A"]/(100*sum_dose["Med B"])
print(breakeven_price_B)
#每單位7.45美元

Med
Med A    792300
Med B      1064
Name: Units, dtype: int64
7.446428571428571


In [208]:
#Q8
med_b_data_Q8=med_b_data.copy()
med_b_data_Q8["Admin Date"]=med_b_data_Q8["Admin Date"].dt.to_period("M")
print(med_b_data_Q8)
print(med_b_data_Q8["ID"].nunique())

category_unit=med_b_data_Q8.groupby("Admin Date")["Units"].sum()
print(category_unit)

category_id=med_b_data_Q8.groupby("Admin Date")["ID"].nunique()
print(category_id)

average_switch=category_unit/category_id
print(average_switch)

       ID    Med Admin Date  Units
26      1  Med B    2012-09      4
27      1  Med B    2012-10      3
28      1  Med B    2012-11      1
82      4  Med B    2012-09      3
83      4  Med B    2012-11      1
...   ...    ...        ...    ...
1979  126  Med B    2012-11      2
2008  128  Med B    2012-09      1
2009  128  Med B    2012-10      1
2010  128  Med B    2012-11      2
2021  129  Med B    2012-09     10

[163 rows x 4 columns]
67
Admin Date
2012-09    448
2012-10    301
2012-11    315
Freq: M, Name: Units, dtype: int64
Admin Date
2012-09    62
2012-10    46
2012-11    51
Freq: M, Name: ID, dtype: int64
Admin Date
2012-09    7.225806
2012-10    6.543478
2012-11    6.176471
Freq: M, dtype: float64


In [261]:
#Q8
admin_Q8=admin_new_Q6.copy()
print(admin_Q8)
admin_Q8["Admin Date"]=admin_Q8["Admin Date"].dt.to_period("M")
sum_Q8=admin_Q8.groupby(["Admin Date","Med"])["Units"].sum()

count_Q8=admin_Q8.groupby(["Admin Date","Med"])["ID"].nunique()
average_Q8=sum_Q8/count_Q8
print(sum_Q8)
print(count_Q8)
print(average_Q8)

       ID    Med Admin Date  Units
24      1  Med A 2012-09-03   1900
25      1  Med A 2012-09-05   1900
26      1  Med B 2012-09-12      4
27      1  Med B 2012-10-17      3
28      1  Med B 2012-11-28      1
...   ...    ...        ...    ...
2009  128  Med B 2012-10-11      1
2010  128  Med B 2012-11-15      2
2019  129  Med A 2012-09-04   5200
2020  129  Med A 2012-09-06   6500
2021  129  Med B 2012-09-13     10

[312 rows x 4 columns]
Admin Date  Med  
2012-09     Med A    792300
            Med B       448
2012-10     Med B       301
2012-11     Med B       315
Name: Units, dtype: int64
Admin Date  Med  
2012-09     Med A    67
            Med B    62
2012-10     Med B    46
2012-11     Med B    51
Name: ID, dtype: int64
Admin Date  Med  
2012-09     Med A    11825.373134
            Med B        7.225806
2012-10     Med B        6.543478
2012-11     Med B        6.176471
dtype: float64


In [ ]:
#Q9


In [69]:
#Q10 9-11月 轉換的病患 反應為lab_b 計算在med_a b 時的平均
#篩選病患  

admin_Q10_d=admin.copy()
admin_Q10_d=admin_Q10_d[(admin_Q10_d["Admin Date"]>="2012-09") & (admin_Q10_d["Admin Date"]<="2012-12")]
print(admin_Q10_d)

labs_Q10=labs_new.copy()

labs_Q10=labs_Q10[(labs_Q10["DRAW_DATE"]>="2012-09")&(labs_Q10["DRAW_DATE"]<="2012-12")]
labs_Q10 = labs_Q10[labs_Q10["LAB_RESULT_CODE"] == "LAB B"]

print(labs_Q10.head(50))
print(admin_Q10_d.head())
print(labs_Q10.nunique())
#完成篩選出3個月內 lab_b的病患



       ID    Med Admin Date  Units
24      1  Med A 2012-09-03   1900
25      1  Med A 2012-09-05   1900
26      1  Med B 2012-09-12      4
27      1  Med B 2012-10-17      3
28      1  Med B 2012-11-28      1
...   ...    ...        ...    ...
2009  128  Med B 2012-10-11      1
2010  128  Med B 2012-11-15      2
2019  129  Med A 2012-09-04   5200
2020  129  Med A 2012-09-06   6500
2021  129  Med B 2012-09-13     10

[387 rows x 4 columns]
       ID  DRAW_DATE LAB_RESULT_CODE LAB_VALUE
10    1.0 2012-09-05           LAB B      10.4
11    1.0 2012-09-19           LAB B      10.7
12    1.0 2012-10-03           LAB B      11.0
13    1.0 2012-10-17           LAB B      11.0
14    1.0 2012-11-07           LAB B      11.9
15    1.0 2012-11-20           LAB B      11.1
33    2.0 2012-09-01           LAB B      10.6
34    2.0 2012-09-04           LAB B      10.0
35    2.0 2012-09-20           LAB B      10.8
36    2.0 2012-10-06           LAB B      13.0
37    2.0 2012-10-18           LAB B   

In [35]:
#1.篩選labs三個月內病患且 lab B
#2.從先前資料取得3個月內病患 
#3-1.分別計算MED A MED B unit
#3-2 計算 lab b 被分類的vakue
print(labs_Q10["DRAW_DATE"].dtypes)


datetime64[ns]


In [105]:
admin_test = admin_Q10_d[(admin_Q10_d["ID"] == 1) & (admin_Q10_d["Med"] == "Med B")]["Admin Date"].min()

labs_test = labs_Q10.copy()

def set_lab_type(row):
    if row["ID"] == 1 and row["DRAW_DATE"] >= admin_test:
        return "Med B"
    else:
        return "Med A"

labs_test["LAB_Type"] = labs_test.apply(set_lab_type, axis=1)
print(labs_test)

         ID  DRAW_DATE LAB_RESULT_CODE LAB_VALUE LAB_Type
10      1.0 2012-09-05           LAB B      10.4    Med A
11      1.0 2012-09-19           LAB B      10.7    Med B
12      1.0 2012-10-03           LAB B      11.0    Med B
13      1.0 2012-10-17           LAB B      11.0    Med B
14      1.0 2012-11-07           LAB B      11.9    Med B
...     ...        ...             ...       ...      ...
2375  128.0 2012-10-04           LAB B      10.9    Med A
2376  128.0 2012-10-18           LAB B      11.3    Med A
2377  128.0 2012-11-08           LAB B      10.1    Med A
2378  128.0 2012-11-19           LAB B       9.2    Med A
2393  129.0 2012-09-04           LAB B       9.4    Med A

[579 rows x 5 columns]


In [112]:
labs_test = labs_Q10.copy()

for patient_id in range(1, 130):
    # 步骤1：在 admin_Q10_d 中筛选 "Med" 为 "Med B" 的日期并选择最小的 "Admin Date"
    admin_test = admin_Q10_d[(admin_Q10_d["ID"] == patient_id) & (admin_Q10_d["Med"] == "Med B")]["Admin Date"].min()
    
    # 步骤2：在 labs_test 中将 "Admin Date" 与 "DRAW_DATE" 进行比较并设置 "LAB_Type"
    def set_lab_type(row):
        if row["ID"] == patient_id and row["DRAW_DATE"] >= admin_test:
            return "Med B"
        else:
            return "Med A"
    
    # 使用 apply 将设置 "LAB_Type" 函数应用于 labs_test
    labs_test["LAB_Type"] = labs_test.apply(set_lab_type, axis=1)
    
    # 输出或保存每个患者 ID 处理后的结果
print(f"Results for Patient ID {patient_id}:")
print(labs_test.head(40))  # 可以根据需求将结果打印或保存到文件



Results for Patient ID 129:
      ID  DRAW_DATE LAB_RESULT_CODE LAB_VALUE LAB_Type
10   1.0 2012-09-05           LAB B      10.4    Med A
11   1.0 2012-09-19           LAB B      10.7    Med A
12   1.0 2012-10-03           LAB B      11.0    Med A
13   1.0 2012-10-17           LAB B      11.0    Med A
14   1.0 2012-11-07           LAB B      11.9    Med A
15   1.0 2012-11-20           LAB B      11.1    Med A
33   2.0 2012-09-01           LAB B      10.6    Med A
34   2.0 2012-09-04           LAB B      10.0    Med A
35   2.0 2012-09-20           LAB B      10.8    Med A
36   2.0 2012-10-06           LAB B      13.0    Med A
37   2.0 2012-10-18           LAB B      12.0    Med A
38   2.0 2012-11-08           LAB B      12.5    Med A
39   2.0 2012-11-19           LAB B      10.6    Med A
59   3.0 2012-09-05           LAB B      11.8    Med A
60   3.0 2012-09-12           LAB B      12.0    Med A
61   3.0 2012-09-19           LAB B      12.1    Med A
62   3.0 2012-10-03           LAB B  